<a href="https://colab.research.google.com/gist/natiska/b53e06518a24b5caae8ab4cc26c3f3b1/delay_all_monitors_in_org.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import getpass
import os
import whylabs_client
from whylabs_client.api import models_api, monitor_api
from whylabs_client.model.list_models_response import ListModelsResponse
from pprint import pprint

In [ ]:
# set your org-id here - should be something like "org-xxxx"
print("Enter your WhyLabs Org ID")
os.environ["WHYLABS_DEFAULT_ORG_ID"] = "org-qEZQUE"

In [ ]:
# set your API key here
print("Enter your WhyLabs API key")
os.environ["WHYLABS_API_KEY"] = getpass.getpass()

print("Using API Key ID: ", os.environ["WHYLABS_API_KEY"][0:10])

In [ ]:
configuration = whylabs_client.Configuration(host="https://api.whylabsapp.com")
configuration.discard_unknown_keys = True

In [ ]:
configuration.api_key['ApiKeyAuth'] = os.environ["WHYLABS_API_KEY"]

In [ ]:
model2monitors = {}

In [ ]:
with whylabs_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    models_api_instance = models_api.ModelsApi(api_client)
    monitor_api_instance = monitor_api.MonitorApi(api_client)
    org_id = os.environ["WHYLABS_DEFAULT_ORG_ID"] # str | Your company's unique organization ID

    # example passing only required values which don't have defaults set
    try:
        # Get a list of all of the model ids for an organization.
        models = models_api_instance.list_models(org_id)
        for item in models["items"]:
            model_id = item['id']
            print(model_id)
            monitors = monitor_api_instance.get_monitor_config_v3(org_id, model_id)
            model2monitors[model_id] = monitors
    except whylabs_client.ApiException as e:
        print("Exception when calling ModelsApi->list_models: %s\n" % e)

## Add dataReadinessDuration parameter to all monitors

In [ ]:
for item in models["items"]:
    model_id = item['id']
    print(model_id)
    monitors = model2monitors[model_id]
    for analyzer in monitors["analyzers"]:
        analyzer["dataReadinessDuration"] = "PT6H"
        response = monitor_api_instance.put_analyzer(org_id, model_id, analyzer["id"], body=analyzer)
        print(response)